## Flow

In [ ]:
# RQA -- in between - raw quality assessment (amplitude * power frequency ratio * alpha band psd ratio)
# Extract file ---> collective data ---> Bad channel removal ----- Not needed [sampling rate correction (500Hz)] -----
# filtering (butterworth bandpass - tune coefficient)  ----- rereferencing ----- epoching ----- ERP ----- baseline removal ---- 

Variables

In [ ]:
SCALINGS = 4e-4
SFREQ = 500
WINDOW_SIZE = 1

Importing all packages

In [ ]:
import numpy as np
import pandas as pd
from scipy import fftpack
from scipy.fft import fft
import time
import mne
import matplotlib
import matplotlib.pyplot as plt
import os
from scipy.stats import kurtosis, zscore
from mne.preprocessing import create_ecg_epochs, create_eog_epochs, read_ica
from mne.time_frequency import tfr_morlet, tfr_array_morlet, morlet, AverageTFR
from itertools import product
import pywt
from scipy.signal import hilbert
from mne_connectivity import spectral_connectivity_epochs
from mne_connectivity.viz import plot_sensors_connectivity
import networkx as nx

matplotlib.use('TkAgg')

## Reading the EEG data

In [ ]:
# folder_path = os.getcwd()+'\\Depression-Sample-dataset-AIIMS\\'
# items = os.listdir(folder_path)
# active_or_sham_list = [item for item in items if os.path.isdir(os.path.join(folder_path, item))]
# for active_or_sham in active_or_sham_list:
#     patient_folder_path = os.path.join(folder_path, active_or_sham)
#     items = os.listdir(patient_folder_path)
#     patients_list = [item for item in items if os.path.isdir(os.path.join(patient_folder_path, item))]
#     patients_list = ['PreetiSingh', 'Hemlata', 'VinodKumarSharma']
#     for patient in patients_list:
#         pre_post_int_folder_path = os.path.join(patient_folder_path, patient)
#         items = os.listdir(pre_post_int_folder_path)
#         pre_post_int_list = [item for item in items if os.path.isdir(os.path.join(pre_post_int_folder_path, item))]
#         for var in pre_post_int_list:
#             if var=='Pre':
#                 pre_path = os.path.join(pre_post_int_folder_path, var)
#                 file_path = pre_path + '\\' + '20230718201550_Preeti singh_22.08.23-01_Eye Close' + '.easy'
#                 break # Remove for all pre, post and intervention for a patient
#         break # Remove for all patients in active or sham
#     break # Remove for both active and sham

In [ ]:
df = pd.read_csv(file_path, sep='\t')
channel_str='Channel 1:P8\
	Channel 2:T8\
	Channel 3:CP6\
	Channel 4:FC6\
	Channel 5:F8\
	Channel 6:F4\
	Channel 7:C4\
	Channel 8:P4\
	Channel 9:AF4\
	Channel 10:Fp2\
	Channel 11:Fp1\
	Channel 12:AF3\
	Channel 13:Fz\
	Channel 14:FC2\
	Channel 15:Cz\
	Channel 16:CP2\
	Channel 17:PO3\
	Channel 18:O1\
	Channel 19:Oz\
	Channel 20:O2\
	Channel 21:PO4\
	Channel 22:Pz\
	Channel 23:CP1\
	Channel 24:FC1\
	Channel 25:P3\
	Channel 26:C3\
	Channel 27:F3\
	Channel 28:F7\
	Channel 29:FC5\
	Channel 30:CP5\
	Channel 31:T7\
	Channel 32:P7'

string_channels = channel_str.replace('\t', ':').split(':')
channel_names = [string_channels[i] for i in range(len(string_channels)) if i % 2 != 0]
channel_names.append('ax')
channel_names.append('ay')
channel_names.append('az')
channel_names.append('trigger')
channel_names.append('timestamp(ms)')
all_channels = np.array(channel_names[:-5])
df.columns=channel_names
df.head()

Data analysis

In [ ]:
transposed_data=df.T

# Create a MNE-Python info object and specifying sampling rate of data
ch_names = df.columns.tolist()[:-5]
ch_types = ['eeg' for i in range(32)]
info = mne.create_info(ch_names=ch_names,ch_types=ch_types, sfreq=500)

# Convert all EEG units to nV
raw = mne.io.RawArray(transposed_data.values[:-5,:]/1e9, info)

print(f"num of channels: {raw.info.get('nchan')}")
print(f'Shape of the data: {raw.get_data().shape}')

Setting custom Montage

In [ ]:
def set_montage(raw):
    mne.channels
    mont1020 = mne.channels.make_standard_montage('standard_1020')
    mont1005 = mne.channels.make_standard_montage('standard_1005')

    ind = [i for (i, channel) in enumerate(mont1020.ch_names) if channel in all_channels]
    mont1020_new = mont1020.copy()

    mont1020_new.ch_names = [mont1020.ch_names[x] for x in ind]
    kept_channel_info = [mont1020.dig[x+3] for x in ind]

    # Keeping the first three rows as they are the fiducial points information
    mont1020_new.dig = mont1020.dig[0:3]+kept_channel_info

    raw.set_montage(mont1020_new)
    mont1020_new.plot()
set_montage(raw)

#### Time amplitude plot

In [ ]:
def time_amplitude(raw, title):
    fig = raw.plot(
        n_channels=32, 
        scalings=SCALINGS
        )
    fig.savefig(f'MNE-graphs/time-amplitude/{title}-EEG.png')

    print(raw.info)
    # TODO: Extract statistical features from time domain such as mean, median, variance, skewness, kurtosis

#### Power spectral density plot

In [ ]:
def psd(raw, title):
    fig = raw.plot_psd(picks=raw.info['ch_names'])
    fig.savefig(f'MNE-graphs/psd-frequency/{title}.png')

#### Wavelet plot

In [ ]:
#### Wavelet plot
def wavelet(raw, title):
    signal = raw.get_data()[0]
    t = np.linspace(0, 299, len(signal))
    coefficients, frequencies = pywt.cwt(signal, scales=np.arange(1, 128), wavelet='cmor')  

    plt.figure(figsize=(10, 6))
    plt.imshow(np.abs(coefficients), aspect='auto', cmap='jet', extent=[0, 299, 1, 128])
    plt.colorbar(label="Magnitude")
    plt.ylabel("Scale")
    plt.xlabel("Time")
    plt.title("CWT")
    plt.show()

#### Raw graph plot

In [ ]:
title = "0--raw graph with line filter"
# time_amplitude(raw, title)
# psd(raw, title)

# Preprocessing

1. Band pass filtering

In [ ]:
def band_pass_filter(raw_cleaned):
    l_freq = 0.01
    h_freq = 45
    raw_cleaned.filter(method= 'fir',
        phase= 'minimum',
        fir_window= 'hann',
        l_freq= l_freq,
        h_freq= h_freq)

    title = "1--Band pass filtered graph"
    time_amplitude(raw_cleaned, title)
    # psd(raw_cleaned, title)
    return raw_cleaned
raw_cleaned  = band_pass_filter(raw.copy())

2. Rereferencing

Calculates the mean voltage from all electrodes at each time point and subtracts this mean from the voltage at each individual electrode.

In [ ]:
def rereferencing(raw_cleaned):
    raw_cleaned.set_eeg_reference('average', projection=True).apply_proj() 

    title = "2---Rereferenced"
    time_amplitude(raw_cleaned, title)
    # psd(raw_cleaned, title)
    return raw_cleaned
raw_cleaned = rereferencing(raw_cleaned)


<!-- 4. ECG/EOG Correction using ICA on Raw data

ICA decomposes data into different components each representing a spatial pattern. Excluding a component means discarding that specific spatial pattern and its contribution to original data -->

3. Artifact Rejection (EOG/ECG) using Wavelet decomposition

In [ ]:
# coeffs = pywt.wavedec(raw_cleaned.get_data(), 'db1', level=2)
# # TODO: Set threshold also don't disturb info field of raw_cleaned - modify this code
# threshold = 0.0000001
# coeffs_thresholded = [pywt.threshold(c, threshold, mode='soft') for c in coeffs]
# denoised_signal = pywt.waverec(coeffs_thresholded, 'db1')

In [ ]:
# title="3---denoised"
# raw_cleaned = mne.io.RawArray(denoised_signal, info)

# time_amplitude(raw_cleaned, title)
# # psd(raw_cleaned, title)